# NO2 Data Aggregation - Plots

Aggregates NO2 values by administrative boundaries and a regular hexagon grid.

To convert from mol/m<sup>2</sup> to molecules/cm<sup>2</sup>, multiply by 6.022140857e+19 (6.022140857 * 10<sup>19</sup>).

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import radiance as rd
import sentinel as stl

In [ ]:
# Set Options
# sns.set(font_scale=1.5, style="whitegrid")
sns.set(font_scale=1.5)
pd.set_option("display.max_columns", None)
pd.set_option("precision", 15)

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

# User-Defined Variables

In [ ]:
# Set paths to Singapore shapefiles at levels 0 and 1, and hexagon grid
singapore_level_0_path = os.path.join(
    "02-raw-data", "vector", "singapore", "gadm36_singapore.shp"
)

singapore_level_1_path = os.path.join(
    "02-raw-data", "vector", "singapore", "gadm36_singapore_level_1.shp"
)

singapore_hexagon_grid_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "singapore_hexagon_grid_0p01.shp",
)

In [ ]:
# Set paths to aggregated data shapfiles
aggregate_level_1_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "aggregate_level_1_jul_2018_jul_2020.shp",
)

aggregate_hexagon_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "aggregate_hexagon_grid_jul_2018_jul_2020.shp",
)

# Data Acquisition and Preprocessing

In [ ]:
# Get geodataframes with geometry only (for plotting)
singapore_level_0_geometry = stl.get_geometry(singapore_level_0_path)
singapore_level_1_geometry = stl.get_geometry(singapore_level_1_path)
singapore_hexagon_grid_geometry = stl.get_geometry(singapore_hexagon_grid_path)

In [ ]:
# Read aggregated shapefiles into geodataframes
singapore_level_1_gdf = gpd.read_file(aggregate_level_1_path)
hexagon_grid_gdf = gpd.read_file(aggregate_hexagon_path)

In [ ]:
# Restore original column names (shorted by export to shapefile)
level_1_gdf = singapore_level_1_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

hexagon_grid_gdf = hexagon_grid_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

# Data Processing

# Data Post-Processing

# Data Visualization

## Level 1 Boundaries

In [ ]:
def plot_aggregate_change(
    aggregated_data,
    magnitude_change,
    percent_change,
    vector_boundary,
    plot_vector=False,
    super_title="Nitrogen Dioxide Change",
    data_source="European Space Agency",
    figsize=(20, 8),
):
    """Plots the NO2 change aggregated a different levels. 
    
    Parameters
    ----------
    aggregated_data : geopandas geodataframe
        Geodataframe containing the aggregated data.
    
    magnitude_change : str
        Column name for the magnitude change.
        
    percent_change : str
        Column name for the percent change.
        
    vector_boundary : geopandas geodataframe
        Geodataframe containing a vector layer to be overlayed.
    
    plot_vector : bool, optional
        Whether or not to plot the vector layer. Default value is False.
    
    super_title : str, optional
        Main plot title. Default value is 'Nitrogen Dioxide Change'.
        
    data_source : str, optional
        Data source. Default value is 'European Space Agency'
    
    Returns
    -------
    tuple

        fig : matplotlib.figure.Figure object
            The figure object associated with the plot.

        ax : matplotlib.axes._subplots.AxesSubplot object
            The axes object associated with the plot.
    
    Example
    -------
        >>>
        >>>
        >>>
        >>>
    """
    # Determine magnitude plot scale
    magnitude_max_absolute = abs(aggregated_data[magnitude_change].max())
    magnitude_min_absolute = abs(aggregated_data[magnitude_change].min())
    magnitude_vmax = (
        magnitude_max_absolute
        if magnitude_max_absolute > magnitude_min_absolute
        else magnitude_min_absolute
    )

    # Plot magnitude and percent change
    with plt.style.context("dark_background"):
        fig, ax = plt.subplots(1, 2, figsize=figsize)
        plt.suptitle(super_title, size=24)

        # Set legend options
        divider_magnitude = make_axes_locatable(ax[0])
        divider_percent = make_axes_locatable(ax[1])
        cax_magnitude = divider_magnitude.append_axes(
            "right", size="5%", pad=-0.25
        )
        cax_percent = divider_percent.append_axes(
            "right", size="5%", pad=-0.25
        )

        # Magnitude change
        aggregated_data.plot(
            column=magnitude_change,
            ax=ax[0],
            legend=True,
            cax=cax_magnitude,
            vmax=magnitude_vmax,
            vmin=-magnitude_vmax,
            cmap="RdBu_r",
            linewidth=0.25,
            edgecolor="gray",
        )

        # Plot vector boundary
        if plot_vector:
            vector_boundary.plot(
                ax=ax[0], facecolor="None", edgecolor="black", linewidth=0.5
            )

        # Configure axes
        ax[0].set_title("Magnitude Change")
        ax[0].set_xlabel("Longitude (degrees)")
        ax[0].set_ylabel("Latitude (degrees)")

        # Percent change
        aggregated_data.plot(
            column=percent_change,
            ax=ax[1],
            legend=True,
            cax=cax_percent,
            cmap="RdBu_r",
            vmin=-100,
            vmax=100,
            linewidth=0.25,
            edgecolor="gray",
        )

        # Plot vector boundary
        if plot_vector:
            vector_boundary.plot(
                ax=ax[1], facecolor="None", edgecolor="black", linewidth=0.5,
            )

        # Configure axes
        ax[1].set_title("Percent Change")
        ax[1].set_xlabel("Longitude (degrees)")
        ax[1].set_ylabel("Latitude (degrees)")

        # Add caption
        fig.text(
            0.5, 0, f"Data Source: {data_source}", ha="center", fontsize=16,
        )

    return fig, ax

In [ ]:
# Plot and export July 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jul_2018_2019_magnitude",
    percent_change="change_jul_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, July 2018 to July 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jul-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export August 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_aug_2018_2019_magnitude",
    percent_change="change_aug_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, August 2018 to August 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-aug-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export September 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_sep_2018_2019_magnitude",
    percent_change="change_sep_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, September 2018 to September 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-sep-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export October 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_oct_2018_2019_magnitude",
    percent_change="change_oct_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapre Nitrogen Dioxide Change, October 2018 to October 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-oct-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export November 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_nov_2018_2019_magnitude",
    percent_change="change_nov_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, November 2018 to November 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-nov-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export December 2018-2019 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_dec_2018_2019_magnitude",
    percent_change="change_dec_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="South Korea Nitrogen Dioxide Change, December 2018 to December 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-dec-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export January 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jan_2019_2020_magnitude",
    percent_change="change_jan_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, January 2019 to January 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jan-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export February 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_feb_2019_2020_magnitude",
    percent_change="change_feb_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, February 2019 to February 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-feb-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export March 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_mar_2019_2020_magnitude",
    percent_change="change_mar_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, March 2019 to March 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-mar-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export April 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_apr_2019_2020_magnitude",
    percent_change="change_apr_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, April 2019 to April 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-apr-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export May 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_may_2019_2020_magnitude",
    percent_change="change_may_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, May 2019 to May 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-may-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export June 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jun_2019_2020_magnitude",
    percent_change="change_jun_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, June 2019 to June 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jun-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export July 2019-2020 change
plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jul_2019_2020_magnitude",
    percent_change="change_jul_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, July 2019 to July 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jul-2019-2020.png",
)

rd.save_figure(outpath)

## Hexagon Grid

In [ ]:
# Plot and export July 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jul_2018_2019_magnitude",
    percent_change="change_jul_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, July 2018 to July 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jul-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export August 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_aug_2018_2019_magnitude",
    percent_change="change_aug_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, August 2018 to August 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-aug-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export September 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_sep_2018_2019_magnitude",
    percent_change="change_sep_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, September 2018 to September 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-sep-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export October 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_oct_2018_2019_magnitude",
    percent_change="change_oct_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, October 2018 to October 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-oct-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export November 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_nov_2018_2019_magnitude",
    percent_change="change_nov_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, November 2018 to November 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-nov-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export December 2018-2019 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_dec_2018_2019_magnitude",
    percent_change="change_dec_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, December 2018 to December 2019",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-dec-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export January 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jan_2019_2020_magnitude",
    percent_change="change_jan_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, January 2019 to January 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jan-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export February 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_feb_2019_2020_magnitude",
    percent_change="change_feb_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, February 2019 to February 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-feb-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export March 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_mar_2019_2020_magnitude",
    percent_change="change_mar_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, March 2019 to March 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-mar-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export April 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_apr_2019_2020_magnitude",
    percent_change="change_apr_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, April 2019 to April 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-apr-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export May 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_may_2019_2020_magnitude",
    percent_change="change_may_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, May 2019 to May 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-may-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export June 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jun_2019_2020_magnitude",
    percent_change="change_jun_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, June 2019 to June 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jun-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export July 2019-2020 change
plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jul_2019_2020_magnitude",
    percent_change="change_jul_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, July 2019 to July 2020",
    data_source="European Space Agency",
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jul-2019-2020.png",
)

rd.save_figure(outpath)

# Data Export